# Working with Raster Bands

## Preparing Your Workspace

### Option 1: (recommended) Run in Google Colab
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kevinlacaille/presentations/blob/main/scipy2024/2_exif.ipynb)

### Option 2: Run local Jupyter instance
You can also choose to open this Notebook in your own local Jupyter instance.

**Prerequisites**

- Install: rasterio, exiftool

In [ ]:
!pip install rasterio
!pip install PyExifTool

In [1]:
import rasterio

In [15]:
image_path = 'data/presentation/8928de8dd83ffff/20213/DJI_0681.JPG'


In [16]:
# import the image
with rasterio.open(image_path) as src:
    # read the entire image
    img = src.read()

/Users/kevinlacaille/.pyenv/versions/3.11.4/envs/scipy2024-env/lib/python3.11/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [4]:
import exiftool
import json

In [17]:
# Extract the metadata from the image
with exiftool.ExifTool() as et:
    metadata = json.loads(et.execute(b'-j', image_path))


In [18]:
# Print the metadata json
print(json.dumps(metadata, indent=4))

[
    {
        "SourceFile": "data/presentation/8928de8dd83ffff/20213/DJI_0681.JPG",
        "ExifTool:ExifToolVersion": 12.6,
        "ExifTool:Warning": "[minor] Possibly incorrect maker notes offsets (fix by 1783?)",
        "File:FileName": "DJI_0681.JPG",
        "File:Directory": "data/presentation/8928de8dd83ffff/20213",
        "File:FileSize": 5224815,
        "File:FileModifyDate": "2024:04:22 11:45:25-07:00",
        "File:FileAccessDate": "2024:07:06 23:20:30-07:00",
        "File:FileInodeChangeDate": "2024:07:06 23:20:29-07:00",
        "File:FilePermissions": 100644,
        "File:FileType": "JPEG",
        "File:FileTypeExtension": "JPG",
        "File:MIMEType": "image/jpeg",
        "File:ExifByteOrder": "II",
        "File:ImageWidth": 4000,
        "File:ImageHeight": 3000,
        "File:EncodingProcess": 0,
        "File:BitsPerSample": 8,
        "File:ColorComponents": 3,
        "File:YCbCrSubSampling": "2 1",
        "EXIF:ImageDescription": "DCIM\\100MEDIA\\D

In [20]:
# Extract the GPS Altitude
altitude = float(metadata[0].get("XMP:RelativeAltitude"))
focal_length = metadata[0].get("EXIF:FocalLength")  # in mm
# Size of pixel = sensor width (m) / image width (px)
image_width = metadata[0].get("File:ImageWidth")
pixel_pitch = 6.17e-3 / image_width  # sensor width known to be 1/2.3” = 6.17mm

print("Altitude: ", altitude, "meters")
print("Focal Length: ", focal_length, "mm")
print("Pixel Pitch: ", pixel_pitch, "meters / px")

Altitude:  87.2 meters
Focal Length:  6.72 mm
Pixel Pitch:  1.5425e-06 meters / px


In [13]:
# Calculate the GSD (Ground Sample Distance)
gsd = (altitude * pixel_pitch) / (focal_length / 1000)
print("GSD: ", gsd, "meters / px")
print("GSD: ", gsd * 100, "cm / px")


GSD:  0.028712994791666672 meters / px
GSD:  2.8712994791666673 cm / px


In [43]:
import numpy as np

height_tree = 10  # meters

# GSD at the tree height
gsd = (altitude - height_tree) * pixel_pitch / focal_length

diameter_of_tree = 10  # meters
# Number of pixels the tree will cover
area_of_tree = diameter_of_tree / gsd
print("Area of tree: ", int(area_of_tree), "pixels")

Area of tree:  378535 pixels
